In [33]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-11-24
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-11-24
@Title : To transform twitter streaming output file into csv file and send it to AWS Redshift

'''

'\n\n@Author: Vighnesh Harish Bilgi\n@Date: 2022-11-24\n@Last Modified by: Vighnesh Harish Bilgi\n@Last Modified time: 2022-11-24\n@Title : To transform twitter streaming output file into csv file and send it to AWS Redshift\n\n'

In [34]:
import boto3
import json
import csv
import pandas as pd

In [35]:
import os
os.environ['AWS_DEFAULT_REGION'] = 'ap-south-1'
os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get('test1_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.environ.get('test1_secret_access_key')

### 1. Fetch the ouptut twitter stream file as a text file and convert into csv file and upload it back into bucket

In [38]:
def connect_to_s3_client():
    """

    Description:
        To connect to AWS S3 service.
    Parameter:
        No parameters
    Return:
        ServiceResource s3
    """
    # s3 =  boto3.resource('s3')
    client = boto3.client("s3")
    return client


def connect_to_s3_resource():
    """

    Description:
        To connect to AWS S3 service through an IAM user.
    Parameter:
        No parameters
    Return:
        ServiceResource s3
    """
    s3 =  boto3.resource(service_name = 's3')
    return s3


def main():

    client = connect_to_s3_client()
    # The file downloaded contains one large string of dictionaries.
    client.download_file('twitter-streaming-output-bucket', 'data/2022/11/24/10/TwitterStreamingDownStream-1-2022-11-24-10-37-32-c7c0825c-23fe-41ff-a2ac-60653e22c842','twitter_sentiment_output.txt')

    with open("twitter_sentiment_output.txt" , encoding="utf8") as file:
        data = file.read()
        list_of_incomp_str_dict = data.split('}')
    
    # print(list_of_incomp_str_dict)

    list_of_comp_str_dict = []

    for inc_dict in list_of_incomp_str_dict:
        list_of_comp_str_dict.append(inc_dict+"}")

    list_of_comp_str_dict.pop()
    # print(list_of_comp_str_dict)

    #Now we convert dictionary as string to dictionary
    list_of_dict = []

    for str_dict in list_of_comp_str_dict:
        list_of_dict.append(json.loads(str_dict))
        # list_of_dict.append(ast.literal_eval(str_dict))

    df = pd.DataFrame(list_of_dict)
    df.to_csv('twitter_sentiment_output.csv', index=False, header=True)


    # creating new bucket 
    s3 = connect_to_s3_resource()
    client.create_bucket(Bucket = 'redshift-twitter-input-bucket',ACL = 'public-read-write',  CreateBucketConfiguration={'LocationConstraint': 'ap-south-1'})
    print("Printing all bucket names to verify if - redshift-twitter-bucket is created:")
    for bucket in s3.buckets.all():
        print(bucket.name)

    # uploading csv file to the new bucket to be accessed by redshift 
    client.upload_file(Filename = 'twitter_sentiment_output.csv',Bucket = 'redshift-twitter-input-bucket', Key = 'twitter_sentiment_output.csv')    

if __name__ == "__main__":
    main()